In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import os
import statsmodels.formula.api as smf
from scipy.stats import pearsonr
from scipy.stats import spearmanr
import statsmodels.api as sm
from scipy.stats.mstats import winsorize

In [2]:
os.chdir(r'/Users/mac/Desktop/Asset Pricing/A shares market/beta/data')
ret = pd.read_csv(os.path.join('TRD_Month.csv'))
daily_data = pd.read_csv(os.path.join('TRD_daily.csv'))
#code = pd.read_csv(os.path.join('Acode.csv'))
ff5 = pd.read_csv(os.path.join('fivefactor_monthly.csv')) #数据来源：央财因子数据库
mktcap = pd.read_csv(os.path.join('fivefactor_daily.csv'))

In [4]:
#定义获取收益和因子数据年份与月份的函数，便于后续处理

def get_month(table,key):
    table[key] = pd.to_datetime(table[key])
    table['year'] = table[key].dt.year
    table['month'] = table[key].dt.month
    return table

def get_month2(table,key):
    table[key] = pd.to_datetime(table[key],format = '%Y%m')
    table['year'] = table[key].dt.year
    table['month'] = table[key].dt.month
    return table

In [140]:
ret.head(10)

,Stkcd,Trdmnt,Opndt,Mopnprc,Clsdt,Mclsprc,Mnshrtrd,Mnvaltrd,Msmvosd,Msmvttl,Ndaytrd,Mretwd,Mretnd,Markettype,Capchgdt,Ahshrtrd_M,Ahvaltrd_M
0,2,1991-01,29,14.58,30,14.51,10000,145000.0,406280.00,598489.62,2,NaN,NaN,4,1991-01-29,NaN,NaN
1,2,1991-02,4,14.66,28,15.09,878500,11065000.0,422520.00,622412.70,13,0.039972,0.039972,4,1991-01-29,NaN,NaN
2,2,1991-03,1,15.01,29,13.18,174500,2996000.0,369040.00,543631.51,21,-0.126574,-0.126574,4,1991-01-29,NaN,NaN
3,2,1991-04,2,13.11,30,11.65,44500,560000.0,326200.00,480524.06,18,-0.116085,-0.116085,4,1991-01-29,NaN,NaN
4,2,1991-05,2,11.59,27,11.29,1550500,17972000.0,316120.00,465675.24,17,-0.030901,-0.030901,4,1991-01-29,NaN,NaN
5,2,1991-06,10,7.50,28,6.55,1626500,10941000.0,370194.03,504156.83,15,-0.303809,-0.303809,4,1991-06-08,NaN,NaN
6,2,1991-07,1,6.60,31,6.15,4955000,24981050.0,347586.76,473368.63,27,-0.061069,-0.061069,4,1991-06-08,NaN,NaN
7,2,1991-08,1,6.15,31,6.30,2725000,15601200.0,356064.49,484914.21,26,0.024390,0.024390,4,1991-06-08,NaN,NaN
8,2,1991-09,2,6.30,30,4.70,4362600,14893175.0,265635.41,361761.39,26,-0.253968,-0.253968,4,1991-06-08,NaN,NaN
9,2,1991-10,3,6.40,31,16.90,10678500,129485325.0,955157.12,1300801.60,25,2.595745,2.595745,4,1991-06-08,NaN,NaN


In [141]:
#计算月度超额收益
'''
**1997-2023月度数据，97年开始的原因是因为，我国A股市场96年实行涨跌停板机制，为了避免影响故从97年开始
**全A股收益率（包含上证A，深A，科创板，创业板）

Trdmnt:股票收益时间，CSMAR字段
trdmn:无风险收益时间，CSMAR字段
'''
ret1 = get_month(ret,'Trdmnt') #获取股票月度收益时间
ff5 = get_month2(ff5,'trdmn') #从因子数据库 ff5 中获取无风险收益的时间

#筛选1997-2023的股票
def time(df,t1,t2):
    df = df[(df['year']>=t1)&(df['year']<=t2)]
    return df
ret1 = time(ret1,1997,2023)
ff5 = time(ff5,1997,2023)
ff = ff5['mkt_rf']
ff.index = ff5['trdmn']

#筛选A股，因为原数据中包含了除主板、创业板和科创板以外的股票，因此需要筛选剔除
def filt(df,x1,x2,x3,x4):
    df = df[(df['Markettype'] == x1)|(df['Markettype'] == x2)|(df['Markettype'] == x3)|(df['Markettype'] == x4)]
    return df

ret2 = filt(ret1,1,4,16,32)

#合并收益率数据计算超额收益

def data(inx,col,value): 
    temp = ff5[['year','month','mkt_rf','rf']]
    df = pd.merge(ret2,temp,on = ['year','month'])   #两个表因为交易时间问题，时间并不一致所以通过年份和月份进行合并 
    df['rt'] = df['Mretnd'] - df['rf']
    month_data = pd.pivot(df,index=inx,columns=col,values=value)
    month_data['month_num'] = (month_data.index.year-1997)*12+month_data.index.month   
    return month_data

month_data = data('Trdmnt','Stkcd','rt')
# data1 = month_data.shift(-1) #这里是为FM回归做的数据准备，可以之后再进行修改
# reg = data1.T  #rt+1
# reg.columns = beta_1m.columns

In [142]:
month_data.head(10)

Stkcd,2,4,6,7,8,9,10,11,12,14,...,688788,688789,688793,688798,688799,688800,688819,688981,689009,month_num
Trdmnt,,,,,,,,,,,,,,,,,,,,,
1997-01-01,0.061791,0.067557,0.091844,0.336260,0.170058,0.050418,0.069247,0.081369,0.092707,0.151873,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1997-02-01,-0.006022,0.146626,-0.047577,0.123978,0.106972,-0.008762,0.168978,0.024977,-0.010470,0.143126,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
1997-03-01,0.278414,0.161546,0.254760,0.435718,0.412760,0.195901,0.500361,0.223377,0.178965,0.252322,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
1997-04-01,0.406234,-0.225929,0.313102,0.323901,-0.113357,-0.127165,0.047650,-0.134645,0.124446,-0.201503,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4
1997-05-01,-0.119927,0.032554,-0.072398,-0.125637,-0.191393,-0.112654,-0.225861,-0.176500,-0.166129,0.260156,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5
1997-06-01,0.349209,0.125407,0.167037,0.149504,0.333461,0.007078,0.095353,0.070419,0.013041,0.052802,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6
1997-07-01,-0.109390,-0.176477,-0.088470,-0.189160,-0.018878,-0.215792,-0.035663,-0.222553,-0.142744,-0.189993,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7
1997-08-01,-0.212124,0.141619,0.166726,-0.104397,-0.152069,0.055796,-0.141091,-0.071401,-0.141185,-0.114281,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8
1997-09-01,-0.175434,-0.190372,-0.142143,-0.192208,-0.223887,-0.119424,-0.088807,-0.155466,-0.162309,-0.220040,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9


In [143]:
daily_data.head(10)

,Unnamed: 0,Stkcd,Trddt,Opnprc,Hiprc,Loprc,Clsprc,Dnshrtrd,Dnvaltrd,Dsmvosd,...,Markettype,Capchgdt,Trdsta,Ahshrtrd_D,Ahvaltrd_D,PreClosePrice,ChangeRatio,LimitDown,LimitUp,LimitStatus
0,0,2,1997-01-02,10.41,10.78,10.30,10.45,2919324,30568143.20,1808900.53,...,4,1996-08-06,1,NaN,NaN,10.47,-0.001910,9.42,11.52,0.0
1,1,2,1997-01-03,10.60,11.20,10.38,10.50,6157916,66558878.15,1817555.55,...,4,1996-08-06,1,NaN,NaN,10.45,0.004785,9.41,11.50,0.0
2,2,2,1997-01-06,10.40,10.50,9.90,9.90,2973721,30129517.64,1713695.24,...,4,1996-08-06,1,NaN,NaN,10.50,-0.057143,9.45,11.55,0.0
3,3,2,1997-01-07,9.70,10.44,9.36,10.24,3794509,38103638.09,1772549.42,...,4,1996-08-06,1,NaN,NaN,9.90,0.034343,8.91,10.89,0.0
4,4,2,1997-01-08,10.24,10.25,9.95,10.04,1845936,18558994.50,1737929.31,...,4,1996-08-06,1,NaN,NaN,10.24,-0.019531,9.22,11.26,0.0
5,5,2,1997-01-09,10.10,10.45,10.10,10.29,2338071,24052645.76,1781204.44,...,4,1996-08-06,1,NaN,NaN,10.04,0.024900,9.04,11.04,0.0
6,6,2,1997-01-10,10.40,10.41,10.10,10.29,1889624,19433797.19,1781204.44,...,4,1996-08-06,1,NaN,NaN,10.29,0.000000,9.26,11.32,0.0
7,7,2,1997-01-13,10.45,10.50,10.30,10.41,2148719,22386630.89,1801976.51,...,4,1996-08-06,1,NaN,NaN,10.29,0.011662,9.26,11.32,0.0
8,8,2,1997-01-14,10.35,10.68,10.20,10.40,2490413,26081140.10,1800245.50,...,4,1996-08-06,1,NaN,NaN,10.41,-0.000961,9.37,11.45,0.0
9,9,2,1997-01-15,10.35,10.45,10.25,10.36,1410712,14605277.90,1793321.48,...,4,1996-08-06,1,NaN,NaN,10.40,-0.003846,9.36,11.44,0.0


In [144]:
#修改daily_data中收益列的名称，Dretwd改为rt
daily_data = daily_data.rename(columns={'Dretwd': 'rt'})

In [145]:
#修改daily_data中的日期列的名称，Trddt改为date
daily_data = daily_data.rename(columns={'Trddt': 'date'})

In [146]:
mktcap.head(10)

,trddy,mkt_rf,smb,hml,umd,rmw,cma,rf,smb_equal,hml_equal,umd_equal,rmw_equal,cma_equal
0,1994-01-04,-0.003950,0.000329,0.003827,0.010374,-0.004755,-0.014706,0.000285,0.002883,0.005370,0.008679,-0.007093,-0.019481
1,1994-01-05,0.007166,0.011850,0.012679,0.009254,-0.020330,-0.006645,0.000285,0.011780,0.013751,0.012208,-0.017225,-0.000045
2,1994-01-06,0.028537,0.012046,0.005722,-0.000346,-0.002358,0.008642,0.000285,0.008664,0.001760,-0.004893,-0.004767,-0.000838
3,1994-01-07,-0.004087,0.009666,0.005175,-0.027683,0.003846,0.004410,0.000285,0.011543,0.007985,-0.028359,0.000227,0.001732
4,1994-01-10,0.002382,0.010774,0.018843,-0.000999,-0.031611,-0.031871,0.000285,0.009054,0.014788,0.001511,-0.025590,-0.010480
5,1994-01-11,-0.012748,-0.003695,-0.005612,0.012066,0.007814,0.003950,0.000285,-0.004199,-0.002248,0.008111,0.002170,-0.002964
6,1994-01-12,-0.005325,0.003344,0.003427,0.002793,0.002738,-0.002812,0.000285,0.007745,0.004416,-0.002070,-0.003284,-0.012985
7,1994-01-13,-0.000292,0.003335,0.010129,-0.008760,-0.007990,-0.008683,0.000285,0.002400,0.010147,-0.004906,-0.005779,0.000705
8,1994-01-14,-0.033700,-0.023248,-0.026024,0.007709,0.037823,0.014779,0.000285,-0.021369,-0.018212,0.007946,0.031944,0.004768
9,1994-01-17,-0.002632,0.015485,0.021263,0.001471,-0.025348,-0.019621,0.000285,0.018612,0.018992,0.000477,-0.023375,-0.006185


In [147]:
#修改mktcap的时间列名，trddy修改为date
mktcap = mktcap.rename(columns={'trddy': 'date'})

In [148]:
#修改daily_data代码列名，Stkcd改为code
daily_data = daily_data.rename(columns={'Stkcd': 'code'})

In [149]:
##筛选日度数据
daily_data = get_month(daily_data,'date')
daily_data = time(daily_data,1997,2023)
daily_data = filt(daily_data,1,4,16,32)
daily_data['month_num'] = (daily_data['year']-1997)*12 + daily_data['month']

##合并收益率数据与无风险利率
mktcap['date'] =  pd.to_datetime(mktcap['date'])
mktcap = mktcap[['date','mkt_rf','rf']]
daily_data = pd.merge(daily_data,mktcap,on='date')
daily_data.head(10)
#daily_data['rt'] = daily_data['rt']-daily_data['rf']  #获得超额收益

# full_data = pd.pivot(daily_data,index='date',columns='code',values='rt')
# full_data['month_num'] = (full_data.index.year-1997)*12+full_data.index.month

# # mktcap2 = mktcap['mkt'] #修改mktcap只剩下mkt数据，但是原数据中只有mkt_rf和rf，可以两个相加得到mkt
# # mktcap2.index = mktcap['date']

,Unnamed: 0,code,date,Opnprc,Hiprc,Loprc,Clsprc,Dnshrtrd,Dnvaltrd,Dsmvosd,...,PreClosePrice,ChangeRatio,LimitDown,LimitUp,LimitStatus,year,month,month_num,mkt_rf,rf
0,0,2,1997-01-02,10.41,10.78,10.30,10.45,2919324,30568143.20,1808900.53,...,10.47,-0.001910,9.42,11.52,0.0,1997,1,1,0.015487,0.000197
1,1196,4,1997-01-02,5.80,6.26,5.71,6.22,1730784,10613037.48,259106.49,...,5.98,0.040134,5.38,6.58,0.0,1997,1,1,0.015487,0.000197
2,2390,6,1997-01-02,13.51,14.01,13.51,13.78,1610052,22096412.00,1162398.73,...,13.59,0.013981,12.23,14.95,0.0,1997,1,1,0.015487,0.000197
3,3591,7,1997-01-02,8.70,9.18,8.52,8.99,887998,7941758.04,366796.44,...,8.94,0.005593,8.05,9.83,0.0,1997,1,1,0.015487,0.000197
4,4787,8,1997-01-02,5.90,6.30,5.90,6.18,1471001,9015815.06,218095.74,...,6.02,0.026578,5.42,6.62,0.0,1997,1,1,0.015487,0.000197
5,5986,9,1997-01-02,6.81,7.00,6.78,6.94,6418291,44121443.37,4019736.83,...,6.91,0.004342,6.22,7.60,0.0,1997,1,1,0.015487,0.000197
6,7185,10,1997-01-02,7.34,7.80,7.30,7.70,1232552,9311546.52,146598.75,...,7.44,0.034946,6.70,8.18,0.0,1997,1,1,0.015487,0.000197
7,8378,11,1997-01-02,7.90,8.20,7.80,8.03,1918575,15338949.28,733872.14,...,8.01,0.002497,7.21,8.81,0.0,1997,1,1,0.015487,0.000197
8,9577,12,1997-01-02,10.10,10.50,10.10,10.21,1842926,18859728.31,819642.31,...,10.23,-0.001955,9.21,11.25,0.0,1997,1,1,0.015487,0.000197
9,10780,14,1997-01-02,5.90,6.35,5.89,6.30,1759039,10869897.64,284888.21,...,6.08,0.036184,5.47,6.69,0.0,1997,1,1,0.015487,0.000197


In [150]:
daily_data['rt'] = daily_data['rt']-daily_data['rf'] 

In [151]:
daily_data.head(5)

,Unnamed: 0,code,date,Opnprc,Hiprc,Loprc,Clsprc,Dnshrtrd,Dnvaltrd,Dsmvosd,...,PreClosePrice,ChangeRatio,LimitDown,LimitUp,LimitStatus,year,month,month_num,mkt_rf,rf
0,0,2,1997-01-02,10.41,10.78,10.30,10.45,2919324,30568143.20,1808900.53,...,10.47,-0.001910,9.42,11.52,0.0,1997,1,1,0.015487,0.000197
1,1196,4,1997-01-02,5.80,6.26,5.71,6.22,1730784,10613037.48,259106.49,...,5.98,0.040134,5.38,6.58,0.0,1997,1,1,0.015487,0.000197
2,2390,6,1997-01-02,13.51,14.01,13.51,13.78,1610052,22096412.00,1162398.73,...,13.59,0.013981,12.23,14.95,0.0,1997,1,1,0.015487,0.000197
3,3591,7,1997-01-02,8.70,9.18,8.52,8.99,887998,7941758.04,366796.44,...,8.94,0.005593,8.05,9.83,0.0,1997,1,1,0.015487,0.000197
4,4787,8,1997-01-02,5.90,6.30,5.90,6.18,1471001,9015815.06,218095.74,...,6.02,0.026578,5.42,6.62,0.0,1997,1,1,0.015487,0.000197


In [152]:
full_data = pd.pivot(daily_data,index='date',columns='code',values='rt')

In [153]:
full_data.head(5)

code,2,4,6,7,8,9,10,11,12,14,...,688787,688788,688789,688793,688798,688799,688800,688819,688981,689009
date,,,,,,,,,,,,,,,,,,,,,
1997-01-02,-0.002107,0.039937,0.013784,0.005396,0.026381,0.004145,0.034749,0.002300,-0.002152,0.035987,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1997-01-03,0.004588,-0.035567,-0.025596,-0.003534,0.087182,-0.011724,-0.052145,0.002294,-0.013909,-0.031943,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1997-01-06,-0.057340,-0.085197,-0.047107,-0.085018,-0.099899,-0.067252,-0.100197,-0.067278,-0.059780,-0.078886,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1997-01-07,0.034146,0.052626,0.026366,0.054681,0.049390,0.045116,0.060686,0.041081,0.025146,0.058522,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1997-01-08,-0.019728,-0.010578,-0.010090,-0.029099,-0.009646,-0.012155,-0.010240,-0.005312,-0.008436,0.033416,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [154]:
full_data['month_num'] = (full_data.index.year-1997)*12+full_data.index.month

In [155]:
full_data.head(5)

code,2,4,6,7,8,9,10,11,12,14,...,688788,688789,688793,688798,688799,688800,688819,688981,689009,month_num
date,,,,,,,,,,,,,,,,,,,,,
1997-01-02,-0.002107,0.039937,0.013784,0.005396,0.026381,0.004145,0.034749,0.002300,-0.002152,0.035987,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1997-01-03,0.004588,-0.035567,-0.025596,-0.003534,0.087182,-0.011724,-0.052145,0.002294,-0.013909,-0.031943,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1997-01-06,-0.057340,-0.085197,-0.047107,-0.085018,-0.099899,-0.067252,-0.100197,-0.067278,-0.059780,-0.078886,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1997-01-07,0.034146,0.052626,0.026366,0.054681,0.049390,0.045116,0.060686,0.041081,0.025146,0.058522,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1997-01-08,-0.019728,-0.010578,-0.010090,-0.029099,-0.009646,-0.012155,-0.010240,-0.005312,-0.008436,0.033416,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [156]:
mktcap['mkt'] = mktcap['mkt_rf'] + mktcap['rf']
mktcap.index = mktcap['date']

In [157]:
mktcap.head(5)

,date,mkt_rf,rf,mkt
date,,,,
1994-01-04,1994-01-04,-0.003950,0.000285,-0.003665
1994-01-05,1994-01-05,0.007166,0.000285,0.007451
1994-01-06,1994-01-06,0.028537,0.000285,0.028822
1994-01-07,1994-01-07,-0.004087,0.000285,-0.003802
1994-01-10,1994-01-10,0.002382,0.000285,0.002667


In [24]:
def beta_calculator(data,factor,span,low_limit):
    '''
    用来计算beta的表格函数，输出是某一种计算方式的beta的表格。
    
    输入参数
    ----------
    data是以month_num为columns，code为index，rt为value
    span是每次回归跨度月份数，一年为12
    low_limit是计算beta的最低样本数（天数），一个月为10，三个月为50等
    输出
    -------
    index为股票代码，columns为月份编号，value为对应规则算出beta 的df
    '''
    X = pd.DataFrame()
    for i in range(max(data['month_num'])-span+1):
        same_time_data = data[(data['month_num']>i)&(data['month_num']<=i+span)]
        same_time = []
        code_list = list(same_time_data.columns[:-1])
        for code in code_list:
            temp_data = same_time_data[code]
            temp_data.name = 'rt'
            reg_data = pd.concat([temp_data,factor],axis=1,join='inner')
            if reg_data['rt'].notna().sum() >= low_limit:
                model = smf.ols('rt~mkt_rf',reg_data,missing='drop').fit()
                beta = model.params[1]
            else:
                beta = np.nan
            same_time.append(beta)
        same_time = pd.Series(same_time,index = code_list,name = i+span)
        X = pd.concat([X,same_time],axis=1)
    return X

In [27]:
# 计算不同时间窗口的 beta
beta_1m = beta_calculator(full_data,mktcap,1,10)
beta_3m = beta_calculator(full_data,mktcap,3,50)
beta_6m = beta_calculator(full_data,mktcap,6,100)
beta_12m = beta_calculator(full_data,mktcap,12,200)
beta_24m = beta_calculator(full_data,mktcap,24,450)
beta_1y = beta_calculator(month_data,ff,12,10)
beta_2y = beta_calculator(month_data,ff,24,20)
beta_3y = beta_calculator(month_data,ff,36,24)
beta_5y = beta_calculator(month_data,ff,60,24)

# 输出beta为csv，方便之后读取（因为算力限制无法直接进行后续计算）
beta_24m.to_csv('beta_24m.csv')
beta_1y.to_csv('beta_1y.csv')
beta_2y.to_csv('beta_2y.csv')
beta_3y.to_csv('beta_3y.csv')
beta_5y.to_csv('beta_5y.csv')

In [158]:
print(daily_data.index.name)
daily_data.head(5)

None


,Unnamed: 0,code,date,Opnprc,Hiprc,Loprc,Clsprc,Dnshrtrd,Dnvaltrd,Dsmvosd,...,PreClosePrice,ChangeRatio,LimitDown,LimitUp,LimitStatus,year,month,month_num,mkt_rf,rf
0,0,2,1997-01-02,10.41,10.78,10.30,10.45,2919324,30568143.20,1808900.53,...,10.47,-0.001910,9.42,11.52,0.0,1997,1,1,0.015487,0.000197
1,1196,4,1997-01-02,5.80,6.26,5.71,6.22,1730784,10613037.48,259106.49,...,5.98,0.040134,5.38,6.58,0.0,1997,1,1,0.015487,0.000197
2,2390,6,1997-01-02,13.51,14.01,13.51,13.78,1610052,22096412.00,1162398.73,...,13.59,0.013981,12.23,14.95,0.0,1997,1,1,0.015487,0.000197
3,3591,7,1997-01-02,8.70,9.18,8.52,8.99,887998,7941758.04,366796.44,...,8.94,0.005593,8.05,9.83,0.0,1997,1,1,0.015487,0.000197
4,4787,8,1997-01-02,5.90,6.30,5.90,6.18,1471001,9015815.06,218095.74,...,6.02,0.026578,5.42,6.62,0.0,1997,1,1,0.015487,0.000197


In [159]:
print(mktcap.index.name)
mktcap.head(5)

date


,date,mkt_rf,rf,mkt
date,,,,
1994-01-04,1994-01-04,-0.003950,0.000285,-0.003665
1994-01-05,1994-01-05,0.007166,0.000285,0.007451
1994-01-06,1994-01-06,0.028537,0.000285,0.028822
1994-01-07,1994-01-07,-0.004087,0.000285,-0.003802
1994-01-10,1994-01-10,0.002382,0.000285,0.002667


In [160]:
# 修改索引名称，否则会出现index与column混淆的情况
mktcap.index.name = 'date1'

In [108]:
def beta_calculator_sw(data, factor):
    '''
    用来计算beta_sw的表格函数，输出是beta_sw计算方式的beta的表格。
    
    输入参数
    ----------
    data是有每只股票每月日度收益和市场超额收益信息的表格，变量命名month_num,code,rt,mkt
    span是每次回归跨度月份数，一年为12
    low_limit是计算beta的最低样本数（天数），一个月为15，三个月为40等
    
    输出
    -------
    index为股票代码，columns为月份编号，value为对应规则算出beta 的df
    '''
    # 创建一个空的DataFrame来存储beta值
    X = pd.DataFrame()

    # 计算市场因子的自相关系数
    rou = pearsonr(factor['mkt'][1:], factor['mkt'][:-1])[0]

    # 为市场超额收益创建不同的时间偏移量
    mktcap1 = mktcap2 = mktcap3 = factor[['date', 'mkt']]
    mktcap1['date'] = mktcap1['date'] + dt.timedelta(days=1)
    mktcap3['date'] = mktcap3['date'] + dt.timedelta(days=-1)

    # 对每个时间窗口进行循环
    for i in range(1, max(data['month_num']) - 12):
        same_time_data = data[(data['month_num'] > i) & (data['month_num'] <= i + 12)]
        same_time = []
        code_list = list(set(same_time_data['code']))

        # 对每只股票分别计算beta值
        for code in code_list:
            temp_data = same_time_data[same_time_data['code'] == code]
            reg_data1 = pd.merge(temp_data, mktcap1, on='date')
            reg_data2 = pd.merge(temp_data, mktcap2, on='date')
            reg_data3 = pd.merge(temp_data, mktcap3, on='date')

            reg_data1 = reg_data1[['rt', 'mkt']]
            reg_data2 = reg_data2[['rt', 'mkt']]
            reg_data3 = reg_data3[['rt', 'mkt']]

            # 检查数据是否足够用来回归
            if reg_data2['rt'].notna().sum() >= 200:
                model1 = smf.ols('rt~mkt', reg_data1, missing='drop').fit()
                model2 = smf.ols('rt~mkt', reg_data2, missing='drop').fit()
                model3 = smf.ols('rt~mkt', reg_data3, missing='drop').fit()

                beta1 = model1.params[1]
                beta2 = model2.params[1]
                beta3 = model3.params[1]

                beta = (beta1 + beta2 + beta3) / (1 + 2 * rou)
            else:
                beta = np.nan

            same_time.append(beta)

        # 将beta值按股票代码索引并加入结果
        same_time = pd.Series(same_time, index=code_list, name=i + 12)
        X = pd.concat([X, same_time], axis=1)

    return X


In [101]:
full_data1 = daily_data[['code','rt','date','month_num']]
beta_sw = beta_calculator_sw(full_data1,mktcap)
# beta_sw.columns = beta_sw.columns+1
beta_sw.to_csv('beta_sw.csv')

/var/folders/nf/zj44wmkd2g9bxsdd_zt1lv580000gn/T/ipykernel_7269/3452621551.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mktcap1['date'] = mktcap1['date'] + dt.timedelta(days=1)
/var/folders/nf/zj44wmkd2g9bxsdd_zt1lv580000gn/T/ipykernel_7269/3452621551.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mktcap3['date'] = mktcap3['date'] + dt.timedelta(days=-1)


In [123]:
full_data.head(5)

code,2,4,6,7,8,9,10,11,12,14,...,688788,688789,688793,688798,688799,688800,688819,688981,689009,month_num
date,,,,,,,,,,,,,,,,,,,,,
1997-01-02,-0.002107,0.039937,0.013784,0.005396,0.026381,0.004145,0.034749,0.002300,-0.002152,0.035987,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1997-01-03,0.004588,-0.035567,-0.025596,-0.003534,0.087182,-0.011724,-0.052145,0.002294,-0.013909,-0.031943,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1997-01-06,-0.057340,-0.085197,-0.047107,-0.085018,-0.099899,-0.067252,-0.100197,-0.067278,-0.059780,-0.078886,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1997-01-07,0.034146,0.052626,0.026366,0.054681,0.049390,0.045116,0.060686,0.041081,0.025146,0.058522,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1997-01-08,-0.019728,-0.010578,-0.010090,-0.029099,-0.009646,-0.012155,-0.010240,-0.005312,-0.008436,0.033416,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [162]:
mktcap.head(5)

,date,mkt_rf,rf,mkt
date1,,,,
1994-01-04,1994-01-04,-0.003950,0.000285,-0.003665
1994-01-05,1994-01-05,0.007166,0.000285,0.007451
1994-01-06,1994-01-06,0.028537,0.000285,0.028822
1994-01-07,1994-01-07,-0.004087,0.000285,-0.003802
1994-01-10,1994-01-10,0.002382,0.000285,0.002667


In [164]:
# mkt只保留mkt_rf进行回归
mktcap1 = mktcap[['mkt_rf']]

In [165]:
mktcap1.head(5)

,mkt_rf
date1,
1994-01-04,-0.003950
1994-01-05,0.007166
1994-01-06,0.028537
1994-01-07,-0.004087
1994-01-10,0.002382


In [166]:
def beta_calculator_d(data, factor):
    '''
    用来计算beta_d的表格函数，输出是beta_d计算方式的beta的表格。
    
    输入参数
    ----------
    data是以month_num为columns，code为index，rt为value
    span是每次回归跨度月份数，一年为12
    low_limit是计算beta的最低样本数（天数），一个月为15，三个月为40等
    
    输出
    -------
    index为股票代码，columns为月份编号，value为对应规则算出beta 的df
    '''
    X = pd.DataFrame()
    mkt = pd.DataFrame()

    for k in range(6):
        x1 = factor.shift(k)
        x2 = factor.shift(-k)
        if k == 0:
            mkt = pd.concat([mkt, x1], axis=1)
        else:
            mkt = pd.concat([mkt, x1], axis=1)
            mkt = pd.concat([mkt, x2], axis=1)

    mkt.columns = ['mkt1', 'mkt2', 'mkt3', 'mkt4', 'mkt5', 'mkt6', 'mkt7', 'mkt8', 'mkt9', 'mkt10', 'mkt11']
    mkt = mkt.dropna()

    for i in range(1, max(data['month_num']) - 12):
        same_time_data = data[(data['month_num'] > i) & (data['month_num'] <= i + 12)]
        same_time = []
        code_list = same_time_data.columns[:-1]

        for code in code_list: #按照股票代码进行循环，计算每只股票的beta
            temp_data = same_time_data[code]
            temp_data.name = 'rt'
            reg_data = pd.concat([temp_data, mkt], axis=1, join='inner')

            if reg_data['rt'].notna().sum() >= 200:
                model = smf.ols('rt ~ mkt6 + mkt5 + mkt7 + mkt4 + mkt8 + mkt3 + mkt9 + mkt2 + mkt10 + mkt1 + mkt11', reg_data, missing='drop').fit()
                beta = sum(model.params[1:])
            else:
                beta = np.nan

            same_time.append(beta)

        same_time = pd.Series(same_time, index=code_list, name=i + 12)
        X = pd.concat([X, same_time], axis=1)
        print(i) #根据循环显示进度

    return X

beta_d = beta_calculator_d(full_data,mktcap1)
# mkt6 + mkt5 + mkt7 + mkt4 + mkt8 + mkt3 + mkt9 + mkt2 + mkt10 + mkt1 + mkt11

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [170]:
beta_d.to_csv('beta_d.csv')

# 描述性统计

In [5]:
beta_1m = pd.read_csv(os.path.join('beta_1m.csv'),index_col=0)
beta_3m = pd.read_csv(os.path.join('beta_3m.csv'),index_col=0)
beta_6m = pd.read_csv(os.path.join('beta_6m.csv'),index_col=0)
beta_12m = pd.read_csv(os.path.join('beta_12m.csv'),index_col=0)
beta_24m = pd.read_csv(os.path.join('beta_24m.csv'),index_col=0)
beta_1y = pd.read_csv(os.path.join('beta_1y.csv'),index_col=0)
beta_2y = pd.read_csv(os.path.join('beta_2y.csv'),index_col=0)
beta_3y = pd.read_csv(os.path.join('beta_3y.csv'),index_col=0)
beta_5y = pd.read_csv(os.path.join('beta_5y.csv'),index_col=0)
beta_sw = pd.read_csv(os.path.join('beta_sw.csv'),index_col=0)
beta_d = pd.read_csv(os.path.join('beta_d.csv'),index_col=0)

In [6]:
def beta_statistic(list_of_beta,name_of_beta):
    X = pd.DataFrame()
    for i in range(len(list_of_beta)):
        x = list_of_beta[i]
        new = pd.Series([x.mean().mean(),x.std().mean(),x.skew().mean(),x.kurt().mean(),x.min().mean(),x.quantile(.05).mean(),x.quantile(.25).mean(),x.median().mean(),x.quantile(.75).mean(),x.quantile(.95).mean(),x.max().mean(),x.count().mean()],
                         index = ['Mean','SD','Skew','Kurt','Min','5%','25%','Median','75%','95%','Max','n'],name = name_of_beta[i])
        X = pd.concat([X,new],axis=1)
    X = X.T
    X = X.applymap(lambda x:round(x, 2))
    return X

df_list = [beta_1m,beta_3m,beta_6m,beta_12m,beta_24m,beta_1y,beta_2y,beta_3y,beta_5y,beta_sw,beta_d]

def drop():
    beta = []
    for i in df_list:
        if i.columns[-1] == '276':
            i = i.drop('276',axis = 1)
        beta.append(i)
    return beta

beta_list = drop()

beta_name_list = ['beta_1m','beta_3m','beta_6m','beta_12m','beta_24m','beta_1y','beta_2y','beta_3y','beta_5y','beta_sw','beta_d']
table1 = beta_statistic(beta_list,beta_name_list)
table1

,Mean,SD,Skew,Kurt,Min,5%,25%,Median,75%,95%,Max,n
beta_1m,1.07,0.80,0.34,118.05,-7.32,0.25,0.78,1.07,1.37,1.88,11.71,2000.11
beta_3m,1.07,0.37,-0.17,24.30,-1.25,0.51,0.86,1.08,1.29,1.62,3.53,1939.52
beta_6m,1.07,0.31,-0.11,17.53,-0.61,0.59,0.90,1.08,1.25,1.52,2.89,1903.36
beta_12m,1.07,0.26,-0.10,14.40,-0.22,0.65,0.93,1.08,1.23,1.44,2.50,1851.88
beta_24m,1.07,0.20,-0.42,8.80,0.15,0.71,0.96,1.08,1.20,1.36,1.86,1582.97
beta_1y,1.07,0.61,0.55,10.28,-2.50,0.17,0.72,1.04,1.39,2.05,5.59,1911.80
beta_2y,1.08,0.43,0.87,14.85,-1.06,0.45,0.82,1.05,1.30,1.77,4.31,1834.96
beta_3y,1.08,0.38,1.18,23.47,-1.11,0.55,0.86,1.06,1.28,1.68,4.35,1894.55
beta_5y,1.09,0.36,1.27,48.87,-2.38,0.61,0.90,1.07,1.26,1.60,5.01,2025.55
beta_sw,2.99,0.72,-0.11,14.26,-0.59,1.81,2.59,3.03,3.43,4.02,6.98,1847.44


# correlation

In [8]:
def personcorr_calculator(dataname1,dataname2):
    X = []
    if len(dataname1.columns)>=len(dataname2.columns):
        month_list = dataname2.columns
    else:
        month_list = dataname1.columns
    for y in month_list:
        x1 = dataname1[y]
        x2 = dataname2[y]
        x = pd.concat([x1,x2],axis=1)
        x = x.dropna(axis=0,how='any')
        person_corr = x.corr('pearson').iloc[0,1]
        X.append(person_corr)
    X = pd.Series(X)
    x = X.mean()
    return x

def spearman_calculator(dataname1,dataname2):
    X = []
    if len(dataname1.columns)>=len(dataname2.columns):
        month_list = dataname2.columns
    else:
        month_list = dataname1.columns
    for y in month_list:
        x1 = dataname1[y]
        x2 = dataname2[y]
        x = pd.concat([x1,x2],axis=1)
        x = x.dropna(axis=0,how='any')
        spearman_corr = x.corr(method = 'spearman').iloc[0,1]
        X.append(spearman_corr)
    X = pd.Series(X)
    x = X.mean()
    return x

In [12]:
def beta_in_list(list_of_beta,name_of_beta):
    ##beta的list顺序和名字顺序要完全对应
    X = pd.DataFrame([],index = name_of_beta,columns = name_of_beta)
    for i in range(len(list_of_beta)):
        for j in range(len(list_of_beta)):
            if i<=j:
                X.iloc[i,j] = spearman_calculator(list_of_beta[i],list_of_beta[j])
            else:
                X.iloc[i,j] = personcorr_calculator(list_of_beta[i],list_of_beta[j])
    X = X.applymap(lambda x:round(x, 2))
    return X

table2 = beta_in_list(beta_list,beta_name_list)            
table2

# Persistance

In [ ]:
def Persistence_calculator(df):
    temp = winsorize(df,limits=(0.005, 0.005))
    temp1 = pd.DataFrame(temp)
    corr = temp1.corr()
    delay_list = [1,3,6,12,24,36,48,60,120]
    X = pd.DataFrame([],index = df.columns,columns = delay_list)
    for x in range(len(df.columns)):
        for y in range(9):
            if x+delay_list[y] < df.shape[1]:
                X.iloc[x,y] = corr.iloc[x,x+delay_list[y]]
    stats = X.mean()
    return stats

def beta_autocorr(list_of_beta,name_of_beta):
    X = pd.DataFrame()
    for i in list_of_beta:
        x = Persistence_calculator(i)
        X = pd.concat([X,x],axis=1)
    del_list = [1,2,3,4,3,4,5,7,3,3]
    for j in range(10):
        k = del_list[j]
        X.iloc[:k,j+1] = np.nan
    X.columns = name_of_beta
    X = X.applymap(lambda x:round(x, 2))
    return X
      
table3 = beta_autocorr(beta_list,beta_name_list)
table3

# Equal-Weighted Portfolios

In [ ]:
monthly_rt = data('Trdmnt','Stkcd','rt').T
monthly_mkt = data('Trdmnt','Stkcd','Msmvttl').T  
monthly_rt.columns = beta_1m.columns
monthly_mkt.columns = beta_1m.columns

ff1 = ff4[['year','month','mkt_rf']]
ff1['month_num'] = (ff1['year']-1997)*12+ff1['month']
ff1 = ff1[['month_num','mkt_rf']].set_index('month_num')

In [ ]:
def capm_reg_equal(beta,rt,mkt_rf):
    mkt_rf.index = rt.columns
    beta_list = pd.DataFrame()
    rt_list = pd.DataFrame()
    for i in beta.columns:
        temp_beta = beta[i]
        temp_rt = rt[i]
        x = pd.concat([temp_beta,temp_rt],axis=1)
        x.columns = ['beta','rt']
        x['group'] = pd.qcut(temp_beta,10,labels=False)
        x = x.dropna()
        beta_avg_i = x.groupby('group')['beta'].mean()
        rt_avg_i = x.groupby('group')['rt'].mean()
        beta_list = pd.concat([beta_list,beta_avg_i],axis=1)
        rt_list = pd.concat([rt_list,rt_avg_i],axis=1)
    beta_list.columns = beta.columns
    rt_list.columns = beta.columns
    beta_list = beta_list.T
    rt_list = rt_list.T
    beta_list.columns = range(1,11)
    rt_list.columns = range(1,11)
    beta_list['10-1'] = beta_list[10] - beta_list[1]
    rt_list['10-1'] = rt_list[10] - rt_list[1]
    alpha = []
    alpha_t = []
    for j in rt_list.columns:
        reg_list = pd.concat([rt_list[j],mkt_rf],axis=1,join='inner')
        reg_list.columns = ['rt','mkt_rf']
        reg_list = reg_list.dropna()
        model = smf.ols('rt~mkt_rf',reg_list).fit(cov_type='HAC',cov_kwds={'maxlags':6})
        alpha.append(model.params[0])
        alpha_t.append(model.tvalues[0])
    beta_avg = beta_list.mean()
    rt_avg = rt_list.mean()
    alpha = pd.Series(alpha,index = beta_avg.index)
    alpha_t = pd.Series(alpha_t,index = beta_avg.index)
    X = pd.concat([beta_avg,rt_avg,alpha,alpha_t],axis=1).T
    X.index = ['beta','excess return','alpha','alpha_t']
    return X

def table4(list_of_beta,rt,mkt_rf):
    X = pd.DataFrame()
    for i in list_of_beta:
        x = capm_reg_equal(i,rt,mkt_rf)
        X = pd.concat([X,x])
    X = X.applymap(lambda x:round(x, 2))
    return X

table4 = table4(beta_list,monthly_rt,ff1)  
table4

# Size-Weighted Portfolios

In [ ]:
def capm_reg_mktweight(beta,rt,mkt,mkt_rf):
    mkt_rf.index = rt.columns
    beta_list = pd.DataFrame()
    rt_list = pd.DataFrame()
    for i in beta.columns:
        temp_beta = beta[i]
        temp_rt = rt[i]
        temp_mkt = mkt[i]
        x = pd.concat([temp_beta,temp_rt,temp_mkt],axis=1)
        x.columns = ['beta','rt','mktcap']
        x['rt*mkt'] = x['rt']*x['mktcap']
        x = x.dropna()
        x['group'] = pd.qcut(temp_beta,10,labels=False)
        beta_avg_i = x.groupby('group')['beta'].mean()
        rt_avg_i = x.groupby('group')['rt*mkt'].sum()/x.groupby('group')['mktcap'].sum()
        beta_list = pd.concat([beta_list,beta_avg_i],axis=1)
        rt_list = pd.concat([rt_list,rt_avg_i],axis=1)
    beta_list.columns = beta.columns
    rt_list.columns = beta.columns
    beta_list = beta_list.T
    rt_list = rt_list.T
    beta_list.columns = range(1,11)
    rt_list.columns = range(1,11)
    beta_list['10-1'] = beta_list[10] - beta_list[1]
    rt_list['10-1'] = rt_list[10] - rt_list[1]
    alpha = []
    alpha_t = []
    for j in rt_list.columns:
        reg_list = pd.concat([rt_list[j],mkt_rf],axis=1,join='inner')
        reg_list.columns = ['rt','mkt']
        model = smf.ols('rt~mkt',reg_list).fit(cov_type='HAC',cov_kwds={'maxlags':6})
        alpha.append(model.params[0])
        alpha_t.append(model.tvalues[0])
    beta_avg = beta_list.mean()
    rt_avg = rt_list.mean()
    alpha = pd.Series(alpha,index = beta_avg.index)
    alpha_t = pd.Series(alpha_t,index = beta_avg.index)
    X = pd.concat([beta_avg,rt_avg,alpha,alpha_t],axis=1).T
    X.index = ['beta','excess return','alpha','alpha_t']
    return X     

capm_reg_mktweight(beta_12m,monthly_rt,monthly_mkt,ff1)

def table5(list_of_beta,rt,mkt,mkt_rf):
    X = pd.DataFrame()
    for i in list_of_beta:
        x = capm_reg_mktweight(i,rt,mkt,mkt_rf)
        X = pd.concat([X,x])
    X = X.applymap(lambda x:round(x, 2))
    return X   

table5 = table5(beta_list,monthly_rt,monthly_mkt,ff1)
table5

# Fama-Macbeth Regression

In [ ]:
def FM_regression1(df1,df2):
    coefs = []
    adj_R = []
    number = []

    for i in df2.columns:       
        temp_beta = df2[i]
        temp_rt = df1[i]
        x = pd.concat([temp_beta,temp_rt],axis=1)
        x.columns = ['beta','rt']
        temp = x.dropna()
        number.append(len(temp)) #样本量
        temp['beta'] = winsorize(temp['beta'], limits=(0.005, 0.005))
        Y = temp['rt']
        X = temp['beta']
        model = sm.OLS(Y.values,sm.add_constant(X).values).fit(cov_type='HAC', cov_kwds={'maxlags': 6})
        coefs.append(model.params)
        adj_R.append(model.rsquared_adj)
    result = pd.DataFrame(coefs)
    result['adj_R'] = adj_R
    result['n'] = number
    
    return result

In [ ]:
def NWtest_1sample(a, lags=6):
    adj_a = np.array(a)
    # 对常数回归
    model = sm.OLS(adj_a, [1] * len(adj_a)).fit(cov_type='HAC', cov_kwds={'maxlags': lags})

    return adj_a.mean(), float(model.tvalues)

In [ ]:
def table6(beta,name):
    temp = pd.DataFrame()
    for i in beta_list:
        data = FM_regression1(reg,i)
        value1 = data.iloc[:, :-2].apply(NWtest_1sample)
        value1 = np.array([list(x) for x in value1.values]).reshape(-1)
        
        value2 = data.iloc[:, -2:].mean().values
        value = pd.Series(list(value1) + list(value2))
        temp = pd.concat([temp,value],axis = 1) 
    temp.index = ['intercept','','beta','','Adj_R2','n']
    temp.columns = name
    return temp

table6 = table6(beta_list,beta_name_list)
table61.iloc[:-1,:] = table6[:-1,:].apply(lambda x:round(x, 3))
table61.iloc[-1,:] = table61.iloc[-1,:].apply(lambda x: int(x)) 
table61